In [11]:
# Import Packages
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
pd.set_option('display.max_colwidth', 100)
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.models import KeyedVectors
import gensim.downloader
from collections import deque

In [12]:
def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model

In [13]:
glove_model = load_glove_model('/Users/andrewsimon/Downloads/glove.6B/glove.6B.300d.txt')

Loading Glove Model
400000 words loaded!


In [14]:
# load csv from zip file
main_data = pd.read_csv('/Users/andrewsimon/Desktop/IMDBDataset.csv.zip')
main_data

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,positive
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",positive
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",positive
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",negative
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",negative
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,negative


In [15]:
# Map sentiment via one hot encoding
main_data['sentiment'] = main_data['sentiment'].map({'positive':1, 'negative': 0})
main_data

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,1
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",1
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,0
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",1
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",1
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",0
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",0
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,0


In [16]:
# Preprocess data for W2v
main_data['review'] = main_data['review'].apply(lambda x: gensim.utils.simple_preprocess(x))
main_data.head()

,review,sentiment
0,"[one, of, the, other, reviewers, has, mentioned, that, after, watching, just, oz, episode, you, ...",1
1,"[wonderful, little, production, br, br, the, filming, technique, is, very, unassuming, very, old...",1
2,"[thought, this, was, wonderful, way, to, spend, time, on, too, hot, summer, weekend, sitting, in...",1
3,"[basically, there, family, where, little, boy, jake, thinks, there, zombie, in, his, closet, his...",0
4,"[petter, mattei, love, in, the, time, of, money, is, visually, stunning, film, to, watch, mr, ma...",1


In [17]:

# Append vectorizations from pre trained model

for i in range(len(main_data['review'])):
    sub_list = []
    for j in main_data['review'][i]:
        try:
            sub_list.append(glove_model[j])
        except KeyError:
            sub_list.append(np.zeros(300,))
    main_data['review'][i] = sub_list

/var/folders/8f/mjj28lvs5fb1m_c_tvmtckl80000gn/T/ipykernel_27821/722170109.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['review'][i] = sub_list


In [18]:
# Separate into training and testing data
X_train, X_test, y_train, y_test = train_test_split(main_data['review'], main_data['sentiment'], test_size=0.2, random_state=1516)

In [19]:
# Pad data to remove ragged arrays
training_padded = pad_sequences(X_train, maxlen=120, truncating='post')

In [20]:

testing_padded = pad_sequences(X_test, maxlen=120, truncating='post')

In [21]:
# Cast to numpy arrays
X_train = X_train.to_numpy()

In [22]:
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [23]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(training_padded, y_train, epochs=num_epochs,batch_size=batch, validation_data=(testing_padded, y_test))

Epoch 1/10


2023-05-09 20:47:43.434522: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1250/1250 [==============================] - 5s 4ms/step - loss: 0.6931 - accuracy: 0.5013 - recall: 0.8233 - auc: 0.4994 - precision: 0.5006 - false_negatives: 3531.0000 - false_positives: 16415.0000 - val_loss: 0.6931 - val_accuracy: 0.4983 - val_recall: 0.0000e+00 - val_auc: 0.5005 - val_precision: 0.0000e+00 - val_false_negatives: 5017.0000 - val_false_positives: 0.0000e+00
Epoch 2/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6635 - accuracy: 0.6012 - recall: 0.5512 - auc: 0.6456 - precision: 0.6119 - false_negatives: 8968.0000 - false_positives: 6985.0000 - val_loss: 0.6487 - val_accuracy: 0.6353 - val_recall: 0.5392 - val_auc: 0.6878 - val_precision: 0.6696 - val_false_negatives: 2312.0000 - val_false_positives: 1335.0000
Epoch 3/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5509 - accuracy: 0.7360 - recall: 0.7531 - auc: 0.7948 - precision: 0.7279 - false_negatives: 4933.0000 - false_positives: 5627.0000 - val_loss: 0.6401 - val_a